数据输入模块

In [16]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# ========== 工具函数 ==========
def safe_exp(x, max_exp=50):
    """对指数函数进行安全计算，避免溢出问题"""
    return np.exp(np.clip(x, -max_exp, max_exp))

def max_env_capacity(base_capacity, growth_rate, year, start_year=2023):
    """计算最大环境容量，考虑增长率"""
    t = year - start_year
    return base_capacity * (1 + growth_rate * t)

# ========== 收入和成本计算函数 ==========
def fixed_tax_revenue():
    """固定税收（例如停泊费）"""
    return 22_000_000

def sales_tax_revenue(N, S, sales_tax_rate=0.05):
    """销售税收入计算"""
    return np.sum(N * S * sales_tax_rate)

def visitor_based_tax(N, S, tau):
    """基于游客的税收收入"""
    return np.sum(np.clip(tau * S * N, 0, 1e12))

def tourism_income(N, S, tau, alpha=0.5, beta=0.001, sales_tax_rate=0.048):
    """计算旅游收入，包括劳动力收入、企业收入和税收"""
    labor_income = np.sum(alpha * N * (1 - np.exp(-beta * N)))
    business_income = np.sum(N * S)
    tax_income = visitor_based_tax(N, S, tau)
    sales_tax = sales_tax_revenue(N, S, sales_tax_rate)
    fixed_tax = fixed_tax_revenue()
    adjusted_tax_revenue = 0.45 * sales_tax + 0.55 * fixed_tax

    return labor_income + business_income + adjusted_tax_revenue

# ========== 分配税收以改善因子 ==========
def allocate_tax_revenue(tax_revenue, P_CO2, F_biodiversity, E_ice_loss, R_residents_satisfaction):
    """根据税收分配资源以改善环境和社会因子"""
    allocation_ratio = 0.3  # 税收分配给环境治理的比例
    environment_funding = tax_revenue * allocation_ratio

    # 改善因子的参数
    P_CO2_reduction = 0.01 * (environment_funding / 1e7)  # CO2排放系数降低
    F_biodiversity_improvement = 0.02 * (environment_funding / 1e7)  # 生物多样性损失降低
    E_ice_loss_reduction = 0.005 * (environment_funding / 1e7)  # 冰川融化降低
    R_residents_satisfaction_improvement = 0.05 * (environment_funding / 1e7)  # 居民满意度提升

    # 更新因子
    P_CO2 = np.maximum(0, P_CO2 - P_CO2_reduction)
    F_biodiversity = np.maximum(0, F_biodiversity - F_biodiversity_improvement)
    E_ice_loss = np.maximum(0, E_ice_loss - E_ice_loss_reduction)
    R_residents_satisfaction = np.minimum(1, R_residents_satisfaction + R_residents_satisfaction_improvement)

    return P_CO2, F_biodiversity, E_ice_loss, R_residents_satisfaction

# ========== 修改环境成本函数 ==========
def environment_cost(N, P_CO2, F_biodiversity, E_ice_loss, tau):
    """环境成本，考虑CO2排放、生物多样性损失、冰川融化"""
    beta1, beta2, beta3 = 0.2, 0.4, 0.3
    return np.sum(beta1 * P_CO2 * N + beta2 * F_biodiversity / (tau + 1) + beta3 * E_ice_loss * N)

# ========== 修改社会成本函数 ==========
def social_cost(N, overcrowding, resident_satisfaction, infrastructure, tau):
    """社会成本，考虑拥挤、居民满意度和基础设施压力"""
    gamma1, gamma2, gamma3 = 0.5, 0.3, 0.2
    return np.sum(gamma1 * overcrowding * N + gamma2 / (tau + 1) * resident_satisfaction + gamma3 * infrastructure * N)

# ========== 输入数据 ==========
path_data = {
    'Path': [1, 2, 3],
    'S_i': [150, 220, 120],  # 提高路径1的消费水平，吸引更多游客
    'P_CO2': [0.06, 0.08, 0.04],  # 降低路径1的CO2排放系数
    'F_biodiversity': [0.1, 0.15, 0.2],
    'E_ice_loss': [0.02, 0.03, 0.02],
    'O_overcrowding': [0.25, 0.3, 0.35],
    'R_residents_satisfaction': [0.8, 0.7, 0.6],
    'I_infrastructure': [0.3, 0.4, 0.5],
    'N_max': [800000, 700000, 900000],  # 每路径游客上限
}
path_df = pd.DataFrame(path_data)

N_max = path_df['N_max'].values
S = path_df['S_i'].values
P_CO2 = path_df['P_CO2'].values
F_biodiversity = path_df['F_biodiversity'].values
E_ice_loss = path_df['E_ice_loss'].values
O_overcrowding = path_df['O_overcrowding'].values
R_residents_satisfaction = path_df['R_residents_satisfaction'].values
I_infrastructure = path_df['I_infrastructure'].values

# ========== 目标函数 ==========
def objective_v2(x, year=2023, base_capacity=20000, growth_rate=0.01, w_env=0.5, w_social=0.3, w_sustain=0.2):
    """改进后的目标函数，平衡收益与可持续发展"""
    N_vars = x[:3]
    tau_vars = x[3:]

    MAX_ENV_CAPACITY = max_env_capacity(base_capacity, growth_rate, year)
    R_tourism = tourism_income(N_vars, S, tau_vars)
    tax_revenue = sales_tax_revenue(N_vars, S) + fixed_tax_revenue() + visitor_based_tax(N_vars, S, tau_vars)

    # 分配税收改善因子
    updated_P_CO2, updated_F_biodiversity, updated_E_ice_loss, updated_R_residents_satisfaction = allocate_tax_revenue(
        tax_revenue, P_CO2, F_biodiversity, E_ice_loss, R_residents_satisfaction
    )

    # 重新计算成本
    C_environment = environment_cost(N_vars, updated_P_CO2, updated_F_biodiversity, updated_E_ice_loss, tau_vars)
    C_social = social_cost(N_vars, O_overcrowding, updated_R_residents_satisfaction, I_infrastructure, tau_vars)
    penalty_env_capacity = 1000 * max(0, np.sum(P_CO2 * N_vars) - MAX_ENV_CAPACITY) ** 2

    # 定义可持续性指标
    sustainability_index = 1 - np.mean([updated_P_CO2, updated_F_biodiversity, updated_E_ice_loss])

    # 平衡权重
    return -(R_tourism - w_env * C_environment - w_social * C_social + w_sustain * sustainability_index - penalty_env_capacity)

# ========== 约束条件 ==========
def adjusted_constraints():
    """生成调整后的约束条件，增加每路径游客的最小限制"""
    cons = [
        {'type': 'ineq', 'fun': lambda x: np.sum(x[:3]) - 1_300_000},  # 总游客数量 ≥ 130万
        {'type': 'ineq', 'fun': lambda x: N_max - x[:3]},  # 每路径游客数量 ≤ 上限
        {'type': 'ineq', 'fun': lambda x: x[:3] - 300_000},  # 每路径游客数量 ≥ 30万
        {'type': 'ineq', 'fun': lambda x: x[3:] - 0.01},  # 税率下限
        {'type': 'ineq', 'fun': lambda x: 0.05 - x[3:]},  # 税率上限
    ]
    return cons

# ========== 初始化值 ==========
def initialize_variables():
    """初始化优化变量"""
    N_init = np.array([600000, 400000, 500000])  # 初始游客分布
    tau_init = np.array([0.03, 0.06, 0.03])         # 初始税率
    return np.concatenate((N_init, tau_init))

# ========== 优化调用 ==========
x0 = initialize_variables()
result = minimize(
    objective_v2, x0, bounds=[(0, 1e6)] * len(x0),
    constraints=adjusted_constraints(), method='trust-constr'
)# ========== 输出优化结果 ==========
if result.success:
    print("优化成功！")
    N_results = result.x[:3]
    tau_results = result.x[3:]

    # 计算各项指标
    tourism_revenue = tourism_income(N_results, S, tau_results)
    environment_costs = environment_cost(N_results, P_CO2, F_biodiversity, E_ice_loss, tau_results)
    social_costs = social_cost(N_results, O_overcrowding, R_residents_satisfaction, I_infrastructure, tau_results)
    tax_revenue = sales_tax_revenue(N_results, S) + fixed_tax_revenue() + visitor_based_tax(N_results, S, tau_results)

    # 分配税收以改善因子
    updated_P_CO2, updated_F_biodiversity, updated_E_ice_loss, updated_R_residents_satisfaction = allocate_tax_revenue(
        tax_revenue, P_CO2, F_biodiversity, E_ice_loss, R_residents_satisfaction
    )

    # 计算可持续化指标
    initial_sustainability_index = 1 - np.mean([P_CO2, F_biodiversity, E_ice_loss])
    updated_sustainability_index = 1 - np.mean([updated_P_CO2, updated_F_biodiversity, updated_E_ice_loss])

    print("\n========= 优化结果 =========")
    print(f"游客分布: 路径1: {N_results[0]:.0f}, 路径2: {N_results[1]:.0f}, 路径3: {N_results[2]:.0f}")
    print(f"税率: 路径1: {tau_results[0]:.2f}, 路径2: {tau_results[1]:.2f}, 路径3: {tau_results[2]:.2f}")
    print(f"旅游收入: {tourism_revenue:.2f}")
    print(f"环境成本: {environment_costs:.2f}")
    print(f"社会成本: {social_costs:.2f}")
    print(f"总税收: {tax_revenue:.2f}")

    print("\n========= 可持续化指标变化 =========")
    print(f"优化前的可持续化指标: {initial_sustainability_index:.4f}")
    print(f"优化后的可持续化指标: {updated_sustainability_index:.4f}")

    print("\n========= 税收对可持续化的贡献 =========")
    print(f"CO2排放降低: {P_CO2} -> {updated_P_CO2}")
    print(f"生物多样性损失降低: {F_biodiversity} -> {updated_F_biodiversity}")
    print(f"冰川融化降低: {E_ice_loss} -> {updated_E_ice_loss}")
    print(f"居民满意度提升: {R_residents_satisfaction} -> {updated_R_residents_satisfaction}")
else:
    print("优化失败！")


优化成功！

========= 优化结果 =========
游客分布: 路径1: 300000, 路径2: 300000, 路径3: 700000
税率: 路径1: 0.05, 路径2: 0.05, 路径3: 0.05
旅游收入: 211962000.00
环境成本: 22700.17
社会成本: 317000.60
总税收: 41410242.60

========= 可持续化指标变化 =========
优化前的可持续化指标: 0.9222
优化后的可持续化指标: 0.9367

========= 税收对可持续化的贡献 =========
CO2排放降低: [0.06 0.08 0.04] -> [0.04757693 0.06757693 0.02757693]
生物多样性损失降低: [0.1  0.15 0.2 ] -> [0.07515385 0.12515385 0.17515385]
冰川融化降低: [0.02 0.03 0.02] -> [0.01378846 0.02378846 0.01378846]
居民满意度提升: [0.8 0.7 0.6] -> [0.86211536 0.76211536 0.66211536]
